In [40]:
import astropy.coordinates as coord
import astropy.table as at
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import scipy.interpolate as sci

import gala.coordinates as gc
from pyia import GaiaData
import galstreams
import cats.photometry as phot

In [18]:
mws = galstreams.MWStreams(verbose=False, implement_Off=True)

Initializing galstreams library from master_log... 


In [19]:
# streams = {
#     'GD-1': 'GD-1-PB18',
#     'Pal5': 'Pal5-PW19',
#     'Jhelum': 'Jhelum-b-B19',
#     'Fjorm-M68': 'M68-P19',
#     'PS1-A': 'PS1-A-B16',
# }

In [49]:
def make_astro_photo_joined_data(gaia_data, phot_data, track6d):
    stream_fr = gs.stream_frame
    
    track = gs.track.transform_to(stream_fr)
    if np.all(track.distance.value == 0):
        raise ValueError(
            "A distance track is required: this stream has no distance information in "
            "the galstreams track."
            
        )
    
    # interpolator to get predicted distance from phi1
    dist_interp = sci.InterpolatedUnivariateSpline(
        track.phi1.degree, track.distance.value, k=1
    )
    
    # get stream coordinates for all stars, and reflex correct with predicted distance
    _c_tmp = gaia_data.get_skycoord(distance=False).transform_to(stream_fr)
    c = gaia_data.get_skycoord(
        distance=dist_interp(_c_tmp.phi1.degree) * track.distance.unit,
        radial_velocity=0*u.km/u.s
    )
    c_stream = c.transform_to(stream_fr)
    c_stream_refl = gc.reflex_correct(c_stream)
    
    # get extinction-corrected photometry and star/galaxy mask
    ext = phot_data.get_ext_corrected_phot()
    ext['star_mask'] = p.get_star_mask()
    
    # start building the final joined table 
    joined = g.data.copy()
    for name in ['phi1', 'phi2', 'pm_phi1_cosphi2', 'pm_phi2']:
        joined[name] = getattr(c_stream_refl, name)
        if name not in ['phi1', 'phi2']:
            joined[f"{name}_unrefl"] = getattr(c_stream, name)
            
    phot_full = at.hstack([phot_data.data, ext])
    cols = ['source_id', 'star_mask'] + [b for b in ext.colnames if b.endswith('0')]
    phot_min = phot_full[cols]
    
    joined = at.join(joined, phot_min, keys='source_id')
    joined = at.unique(joined, keys='source_id')
    
    return joined

# GD-1

In [21]:
ms_name = 'GD-1-PB18'
g = GaiaData('../data/GaiaDR3-GD-1-all.fits')
p = phot.PS1Phot('../data/PS1DR2-GD-1_xm.fits')
gs = mws[ms_name]

In [52]:
joined = make_astro_photo_joined_data(g, p, gs)
# joined.write('../data/GD-1-joined.fits', overwrite=True)

KeyboardInterrupt: 

# Pal 5

In [ ]:
ms_name = 'Pal5-PW19'
g = GaiaData('../data/GaiaDR3-')
p = phot.PS1Phot('../data/PS1DR2-GD-1_xm.fits')
gs = mws[ms_name]

In [ ]:
joined = make_astro_photo_joined_data(g, p, gs)
joined.write('../data/GD-1-joined.fits', overwrite=True)

# Jhelum

In [ ]:
phot.DESY6('../data/Jhelum/jhelum_desy6.fits')

# PS1-A

# Fjorm

# PS1 test:

In [ ]:
from cats.photometry import PS1

In [3]:
ps1 = PS1('../data/PS1DR2-PS1-A_xm.fits')

In [8]:
ext = ps1.get_ext_corrected_phot()
star_mask = ps1.get_star_mask()

ext[:3]

A_gMeanPSFMag,gMeanPSFMag_0,A_rMeanPSFMag,rMeanPSFMag_0,A_iMeanPSFMag,iMeanPSFMag_0,A_zMeanPSFMag,zMeanPSFMag_0,A_yMeanPSFMag,yMeanPSFMag_0
float32,float64,float32,float64,float32,float64,float32,float64,float32,float64
0.060273785,16.824726443737745,0.04315314,16.541947033256292,0.03196107,16.44293822348118,0.025120413,16.44757970608771,0.020654984,16.420545272529125
0.06000212,21.573298662900925,0.04295864,20.57894191890955,0.03181701,19.97248215228319,0.025007188,19.750692427158356,0.020561887,19.541337298229337
0.0593879,20.078511498868465,0.04251889,19.656980238854885,0.031491317,19.49820800870657,0.024751201,19.41864822804928,0.020351402,19.335248954594135


# Gaia:

In [1]:
from cats.photometry import GaiaDR3

In [2]:
gaia = GaiaDR3('../data/GaiaDR3-Jhelum-all.fits')

In [3]:
ext = gaia.get_ext_corrected_phot()

In [4]:
ext[:3]

A_G,G0,A_BP,BP0,A_RP,RP0
float32,float32,float32,float32,float32,float32
0.0435073,16.323257,0.05504485,16.640823,0.030577885,15.849213
0.043066513,16.685736,0.05455922,17.006218,0.030336939,16.198462
0.035442032,19.337635,0.052523,20.453585,0.029214298,18.329704


# DES test:

In [1]:
from cats.photometry import DESY6

In [2]:
des = DESY6('../data/Jhelum/jhelum_desy6.fits')

In [3]:
ext = des.get_ext_corrected_phot()

In [4]:
ext[:3]

A_g,g0,A_r,r0
float32,float64,float32,float64
0.0524416,20.175627147879865,0.035252683,19.024862682415737
0.05348761,19.689535929914882,0.03595584,19.110654449832712
0.054753695,24.834807728067002,0.036806934,24.72642080687828
